In [34]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import numpy as np

In [35]:
# --- 1. Dataset Class ---
class SpicyFoodDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.data = self.data[self.data['spice_label'] != -1].reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        url = self.data.loc[idx, 'image_url']
        label = int(self.data.loc[idx, 'spice_label'])

        try:
            response = requests.get(url, timeout=10)
            img = Image.open(BytesIO(response.content)).convert('RGB')
        except:
            # If download fails, return black image with label 0
            img = Image.new('RGB', (224, 224))
            label = 0

        if self.transform:
            img = self.transform(img)

        return img, label

In [36]:
# --- 2. Transformations ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# --- 3. Load Dataset & DataLoader ---
dataset = SpicyFoodDataset("data/sample-data.csv", transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# --- 4. Load Pretrained Model ---
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 6)  # 6 spice levels: 0 to 5

# --- 5. Loss & Optimizer ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


/Users/shusei/workspace/HowHot/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/shusei/workspace/HowHot/.venv/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [37]:
# --- 6. Training Loop ---
EPOCHS = 7
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss:.4f}")

Epoch [1/5], Loss: 18.4915
Epoch [2/5], Loss: 6.3314
Epoch [3/5], Loss: 2.7900
Epoch [4/5], Loss: 2.3153
Epoch [5/5], Loss: 1.0973


In [29]:
# --- 7. Save Model ---
torch.save(model.state_dict(), "model/spice_model.pth")

In [33]:
# Replace with your test image URL
image_url = "https://fastly.4sqi.net/img/general/original/96559263_BcpMkB9VmdHruG3zEFMwlIgmvYa2JK8lPVq8y6sL2EU.jpg"

response = requests.get(image_url)
img = Image.open(BytesIO(response.content)).convert('RGB')
img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

# Predict spice level
with torch.no_grad():
    output = model(img_tensor)
    predicted_spice = torch.argmax(output, dim=1).item()

print(f"Predicted spice level: {predicted_spice} (scale 0–5)")


Predicted spice level: 0 (scale 0–5)
